In [1]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv


In [2]:
BASE_URL = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0"
VILAGE_BASE_TIMES = ["0200", "0500", "0800", "1100", "1400", "1700", "2000", "2300"]


In [3]:
def load_service_key():
    load_dotenv()
    key = os.getenv("KMA_SERVICE_KEY")
    if not key:
        raise ValueError("KMA_SERVICE_KEY가 .env에서 안 읽힘")
    return key


def pick_latest_vilage_base(dt_kst=None):
    KST = timezone(timedelta(hours=9))
    if dt_kst is None:
        dt_kst = datetime.now(KST)

    ymd = dt_kst.strftime("%Y%m%d")
    hm = dt_kst.strftime("%H%M")

    candidates = [t for t in VILAGE_BASE_TIMES if t <= hm]
    if candidates:
        return ymd, candidates[-1]

    # 새벽 시간대 → 전날 23시
    ymd_yesterday = (dt_kst - timedelta(days=1)).strftime("%Y%m%d")
    return ymd_yesterday, "2300"


In [4]:
def fetch_vilage_items_all(service_key, nx, ny, base_date, base_time,
                           num_rows=1000, max_pages=50):

    url = f"{BASE_URL}/getVilageFcst"
    all_items = []
    total_count = None

    for page in range(1, max_pages + 1):
        params = {
            "serviceKey": service_key,
            "numOfRows": num_rows,
            "pageNo": page,
            "dataType": "JSON",
            "base_date": base_date,
            "base_time": base_time,
            "nx": int(nx),
            "ny": int(ny),
        }

        r = requests.get(url, params=params, timeout=20)
        r.raise_for_status()

        data = r.json()
        header = data["response"]["header"]
        if header["resultCode"] != "00":
            raise RuntimeError(header["resultMsg"])

        body = data["response"]["body"]
        if total_count is None:
            total_count = body.get("totalCount")

        items = body["items"]["item"]
        all_items.extend(items)

        if total_count and len(all_items) >= total_count:
            break

    return all_items


In [5]:
def preprocess_items(items, airport_name, nx, ny, base_date, base_time):
    df = pd.DataFrame(items)
    if df.empty:
        return pd.DataFrame()

    df = df[df["category"].isin(["TMP", "WSD"])].copy()
    df["fcst_datetime"] = pd.to_datetime(
        df["fcstDate"] + df["fcstTime"],
        format="%Y%m%d%H%M"
    )

    wide = (
        df.pivot_table(
            index="fcst_datetime",
            columns="category",
            values="fcstValue",
            aggfunc="first"
        )
        .reset_index()
        .rename(columns={"TMP": "temp_c", "WSD": "wind_speed_ms"})
    )

    wide["temp_c"] = pd.to_numeric(wide["temp_c"], errors="coerce")
    wide["wind_speed_ms"] = pd.to_numeric(wide["wind_speed_ms"], errors="coerce")

    wide["공항"] = airport_name
    wide["nx"] = nx
    wide["ny"] = ny
    wide["base_date"] = base_date
    wide["base_time"] = base_time

    return wide[
        ["공항", "fcst_datetime", "temp_c", "wind_speed_ms",
         "nx", "ny", "base_date", "base_time"]
    ]


In [6]:
def build_airport_forecast_df(airport_csv_path="airport_nxny_map.csv"):
    service_key = load_service_key()
    base_date, base_time = pick_latest_vilage_base()

    ap = pd.read_csv(airport_csv_path)

    required_cols = {"공항", "nx", "ny"}
    if not required_cols.issubset(ap.columns):
        raise ValueError(f"CSV 컬럼 필요: {required_cols}")

    out = []

    for _, r in ap.iterrows():
        items = fetch_vilage_items_all(
            service_key,
            r["nx"],
            r["ny"],
            base_date,
            base_time
        )

        df_one = preprocess_items(
            items,
            r["공항"],
            r["nx"],
            r["ny"],
            base_date,
            base_time
        )

        out.append(df_one)

    df_all = (
        pd.concat(out, ignore_index=True)
        .sort_values(["공항", "fcst_datetime"])
        .reset_index(drop=True)
    )

    return df_all


In [7]:
def get_weather_at_time_by_airport_name(df_all, airport_name, target_dt):
    if isinstance(target_dt, str):
        target_dt = pd.to_datetime(target_dt)

    df_ap = df_all[df_all["공항"] == airport_name]
    if df_ap.empty:
        raise ValueError(f"공항명 '{airport_name}' 데이터 없음")

    df_before = df_ap[df_ap["fcst_datetime"] <= target_dt]
    if df_before.empty:
        return None

    row = df_before.sort_values("fcst_datetime").iloc[-1]

    return {
        "공항": row["공항"],
        "조회시각": target_dt,
        "기준예보시각": row["fcst_datetime"],
        "기온(°C)": row["temp_c"],
        "풍속(m/s)": row["wind_speed_ms"],
        "base_date": row["base_date"],
        "base_time": row["base_time"],
    }


In [8]:
# 1️⃣ 예보 데이터 생성
df_all = build_airport_forecast_df("airport_nxny_map.csv")

display(df_all.head(20))


category,공항,fcst_datetime,temp_c,wind_speed_ms,nx,ny,base_date,base_time
0,광주,2026-01-26 18:00:00,0,2.5,57,74,20260126,1700
1,광주,2026-01-26 19:00:00,0,1.8,57,74,20260126,1700
2,광주,2026-01-26 20:00:00,-1,3.1,57,74,20260126,1700
3,광주,2026-01-26 21:00:00,-2,2.8,57,74,20260126,1700
4,광주,2026-01-26 22:00:00,-2,2.5,57,74,20260126,1700
5,광주,2026-01-26 23:00:00,-3,2.7,57,74,20260126,1700
6,광주,2026-01-27 00:00:00,-3,2.8,57,74,20260126,1700
7,광주,2026-01-27 01:00:00,-3,2.3,57,74,20260126,1700
8,광주,2026-01-27 02:00:00,-3,2.0,57,74,20260126,1700
9,광주,2026-01-27 03:00:00,-3,2.3,57,74,20260126,1700


In [9]:
# 2️⃣ 사용자 입력 예시
result = get_weather_at_time_by_airport_name(
    df_all,
    airport_name="김포",
    target_dt="2026-01-27 19:00"
)

result


{'공항': '김포',
 '조회시각': Timestamp('2026-01-27 19:00:00'),
 '기준예보시각': Timestamp('2026-01-27 19:00:00'),
 '기온(°C)': -7,
 '풍속(m/s)': 3.4,
 'base_date': '20260126',
 'base_time': '1700'}